In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_community.utilities import SQLDatabase

db_path = "../products.db"

db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

In [4]:
print(db.dialect)

sqlite


In [5]:
print(db.get_usable_table_names())

['products']


In [6]:
db.run("SELECT * FROM products;")

"[(1, 'I Phone 15 pro max', 'http://res.cloudinary.com/dujmaxztw/image/upload/v1710849551/dypzg5qwma5k2t24nr9c.jpg', 'This is the latest I phone 15 pro max mobile', 2, 190000.0), (2, 'Redmi Note 9 pro', 'http://res.cloudinary.com/dujmaxztw/image/upload/v1710849551/dypzg5qwma5k2t24nr9c.jpg', 'This is redmi note 9 pro latest global version mobile', 9, 20000.0), (3, 'calculator', 'http://res.cloudinary.com/dujmaxztw/image/upload/v1710849551/dypzg5qwma5k2t24nr9c.jpg', 'casio fx caculator', 2, 1200.0), (4, 'Samsung Galaxy M30', 'https://ibb.co/sbLrxnm', 'This is samsung galaxy m30 smartphone.', 5, 20000.0), (5, 'Samsung Galaxy S22', 'https://ibb.co/Svdqcpb', 'This is samsung galaxy S22', 5, 200000.0), (6, 'Samsung Galaxy tab', 'https://ibb.co/YcfZcfm', 'This is samsung galaxy tab', 5, 110000.0), (7, 'Infinix smart 8 pro', 'https://ibb.co/s5YVpnw', 'Infinix smart 8 pro device for gaming.', 3, 30000.0), (8, 'Samsung S23 Ultra', 'https://ibb.co/DwFxTND', 'Samsung s23 ultra phone titanium.', 3,

In [7]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192")

In [8]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(model, db)

In [9]:
chain

RunnableAssign(mapper={
  input: RunnableLambda(...),
  table_info: RunnableLambda(...)
})
| RunnableLambda(lambda x: {k: v for k, v in x.items() if k not in ('question', 'table_names_to_use')})
| PromptTemplate(input_variables=['input', 'table_info'], partial_variables={'top_k': '5'}, template='You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below. B

In [10]:
response = chain.invoke({"question": "How many products are there"})

In [11]:
chain.invoke({
    "question": "what is the highest price of your product"
})

'Question: what is the highest price of your product\nSQLQuery: SELECT "price" FROM products ORDER BY "price" DESC LIMIT 1;'

In [12]:
chain.invoke({
    "question": "Do you have I phone 15 pro"
})

'Here is the answer:\n\nQuestion: Do you have I phone 15 pro\nSQLQuery: SELECT * FROM products WHERE "name" LIKE "%I Phone 15 pro%"'

In [13]:
chain.invoke({
    "question": "What is the price of redmi note 9 pro mobile phone"
})

'Here is the answer:\n\nQuestion: What is the price of redmi note 9 pro mobile phone\nSQLQuery: SELECT "price" FROM products WHERE "name" = \'Redmi Note 9 pro\''

In [14]:
print(chain.invoke({"question": "How many products are there"}))

Question: How many products are there
SQLQuery: SELECT COUNT(*) AS "Count" FROM products


In [15]:
import re

def extract_query(text):
    # Define the regular expression to find the SQL query part
    pattern = r'SQLQuery:\s*(.*)'
    match = re.search(pattern, text)
    
    if match:
        return match.group(1)
    else:
        return None

In [16]:
query = extract_query(response)

In [17]:
db.run(query)

'[(9,)]'

In [18]:
print(response)

Question: How many products are there
SQLQuery: SELECT COUNT(*) AS num_products FROM products


In [19]:
from langchain_core.runnables import RunnableLambda

query_extractor = RunnableLambda(lambda x: extract_query(x))

In [20]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query  = create_sql_query_chain(model, db)
execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | query_extractor | execute_query

In [21]:
chain

RunnableAssign(mapper={
  input: RunnableLambda(...),
  table_info: RunnableLambda(...)
})
| RunnableLambda(lambda x: {k: v for k, v in x.items() if k not in ('question', 'table_names_to_use')})
| PromptTemplate(input_variables=['input', 'table_info'], partial_variables={'top_k': '5'}, template='You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below. B

In [22]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [23]:
chain.invoke({"question": "How many products are there"})

'[(9,)]'

In [24]:
from operator import itemgetter
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [25]:
answer_prompt = PromptTemplate.from_template(
    """
    Given the following user question, corresponding  SQL query, and the result of the query, answer the question in a short and concise manner.
    Question: {question}
    SQL Query: {query}
    SQL Result: {result}
    Answer:
    """
)

answer = answer_prompt | model | StrOutputParser()
answer

PromptTemplate(input_variables=['query', 'question', 'result'], template='\n    Given the following user question, corresponding  SQL query, and the result of the query, answer the question in a short and concise manner.\n    Question: {question}\n    SQL Query: {query}\n    SQL Result: {result}\n    Answer:\n    ')
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000263F9C07F20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000263FAD2A570>, model_name='llama3-8b-8192', groq_api_key=SecretStr('**********'))
| StrOutputParser()

In [26]:
query_getter = write_query | query_extractor

In [27]:
final_chain = (
    RunnablePassthrough.assign(query=query_getter).assign(result=itemgetter("query") | execute_query)
    | answer
)

In [28]:
final_chain

RunnableAssign(mapper={
  query: RunnableAssign(mapper={
           input: RunnableLambda(...),
           table_info: RunnableLambda(...)
         })
         | RunnableLambda(lambda x: {k: v for k, v in x.items() if k not in ('question', 'table_names_to_use')})
         | PromptTemplate(input_variables=['input', 'table_info'], partial_variables={'top_k': '5'}, template='You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\

In [29]:
response = final_chain.invoke({"question": "How many products are there"})

In [30]:
print(response)

There are 9 products.


In [31]:
print(final_chain.invoke({"question": "Do you have Redmi note 9 pro mobile"}))

Yes, we have Redmi Note 9 Pro mobile.


In [32]:
print(final_chain.invoke({"question": "What is the price of redmi note 9 pro mobile phone"}))

The price of the Redmi Note 9 Pro mobile phone is 20000.
